In [44]:
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
# ^^^ pyforest auto-imports - don't write above this line
import pyforest

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import matplotlib.pyplot as pt
import seaborn

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelBinarizer

#modeles :
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

import warnings
import os

warnings.filterwarnings("ignore")

In [45]:
path = 'D:\\Etude_Data_science\\Kaggle_competition\\03_Multi-Class Prediction of Obesity Risk'+"\\dataset\\"
#chargement des données d'entrainement sans preprocessing : 
data = pd.read_csv(path+"train.csv")

#Chargement des données d'entrainement après preprocessing : 
final_train = pd.read_csv(path+"final_df.csv")

#Chargement du fichier test :
test = pd.read_csv(path+"test.csv")

### Chargement des preprocessing : 

In [46]:
with open('ohe.pkl', 'rb') as f:
    ohe = pickle.load(f)
with open('ordinalencoder.pkl', 'rb') as f:
    ordinalencoder = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('target_encoder.pkl', 'rb') as f:
    target_encoder = pickle.load(f)
    

## Preprocessing du fichier test :

### Premier preprocessing :

In [47]:
def preprocessing(data):
    """fonction qui resume les étapes de préprocessing obtenues lors de l'EDA"""
    var_cont = list(data.select_dtypes(exclude="object").columns)
    var_qual =  list(data.select_dtypes(include="object").columns)
    var_nom = ["MTRANS"]
    var_ord = [i for i in data.select_dtypes("object") if i not in var_nom ]
    
    var_dis = ["FCVC", "NCP", "CH2O", "FAF", "TUE"]
    #Arrondinr les valeurs :
    data[var_dis] = data[var_dis].apply(lambda x:round(x))

    #Convertir les colonnes en int :
    data = data.astype({col: int for col in var_dis})
    var_cat = var_dis+var_qual
    
    #Supprimons les variables qu'on considère discrètes de var_cont :
    var_cont = [i for i in var_cont if i not in var_dis]
    return var_cont, var_qual, var_nom,var_ord, var_dis,var_cat, data

In [48]:
var_cont, var_qual, var_nom,var_ord, var_dis,var_cat, preprocessed_test = preprocessing(test)

In [49]:
preprocessed_test.head() #données après premier prétraitement 

,id,Gender,Age,Height,Weight,...,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,...,no,1,0,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,...,no,1,0,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,...,no,0,0,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,...,no,0,0,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,...,no,0,1,Sometimes,Public_Transportation


In [50]:
var_cont.remove("id") #retirons l"id

### StandardScaler :

In [51]:
def standardisation(data, columns):
    """fonction pour laquelle on rentre :
    data : dataframe
    columns : colonnes sur lequel appliqué un StandardScaler
    On rentre un dataframe avec les colonnes a standardisées et renvoie le dataframe standardisé"""
    data_scaled = scaler.transform(data[columns])
    df_scaled = pd.DataFrame(data_scaled, columns=columns, index = data.index)
    return df_scaled

In [52]:
preprocessed_test[var_cont] = standardisation(preprocessed_test, var_cont)

### OrdinalEncoder : 

In [53]:
print("Dans les données d'entrainement :\n")
for i in var_ord:
    print(f"{i:-<20} : {data[i].unique()}")
print()
    
print("Dans les données test :\n")
for i in var_ord:
    print(f"{i:-<20} : {preprocessed_test[i].unique()}")


Dans les données d'entrainement :

Gender-------------- : ['Male' 'Female']
family_history_with_overweight : ['yes' 'no']
FAVC---------------- : ['yes' 'no']
CAEC---------------- : ['Sometimes' 'Frequently' 'no' 'Always']
SMOKE--------------- : ['no' 'yes']
SCC----------------- : ['no' 'yes']
CALC---------------- : ['Sometimes' 'no' 'Frequently']

Dans les données test :

Gender-------------- : ['Male' 'Female']
family_history_with_overweight : ['yes' 'no']
FAVC---------------- : ['yes' 'no']
CAEC---------------- : ['Sometimes' 'Always' 'Frequently' 'no']
SMOKE--------------- : ['no' 'yes']
SCC----------------- : ['no' 'yes']
CALC---------------- : ['Sometimes' 'no' 'Frequently' 'Always']


On peut apercevoir que pour la colonne **CALC** dans les données d'entrainement nous avons 3 valeurs différentes, et dans les données tests une nouvelles valeurs est trouvées **Always**. D'où l'intérêt d'avoir préciser dans le notebook 01_EDA :
**handle_unknown="use_encoded_value** et **unknown_value=-1** qui signifie que si notre encoder trouve des données non présente à l'entrainement, celles-ci seront remplacées par la valeur -1

In [54]:
def ordinalisation(data, columns):
    """fonction pour laquelle on rentre :
    data : dataframe
    columns : colonnes sur lequel appliqué un OrdinalEncoder
    fonction qui encode les variables ordinales en données numériques"""
    data_transformed = ordinalencoder.transform(data[columns])
    df_transformed = pd.DataFrame(data_transformed, columns=columns, index = data.index)
    return df_transformed

In [55]:
preprocessed_test[var_ord] = ordinalisation(preprocessed_test, var_ord)

### OneHotEncoding :

In [56]:
def onehot(data,columns):
    """fonction qui renvoi un dataframe après labelbinarizer
    data : données à binarizer
    columns : colonnes sur lequel appliquer le binarizer
    index_df : les numéros de lignes obtenues après la création du nouveau dataframe"""
    data_transformed = ohe.transform(data[columns])
    data_transformed_df = pd.DataFrame(data = data_transformed, columns=ohe.classes_, index=data.index)
    return data_transformed_df

In [57]:
def concat_data(df1,df2,col_a_suppr):
    """Fonction qui permet de fusionner sur l'axe 1 deux dataframes
    et qui supprimes les anciennes colonnes sur le premier dataframe"""
    df1_temp = df1.drop(col_a_suppr, axis = 1,)
    new_df = pd.concat([df1_temp,df2], axis = 1)
    return new_df

In [58]:
df2 = onehot(preprocessed_test, var_nom) #création d'un dataframe "fille" contenant une colonne pour chaque valeur trouvée
#dans la colonne mère MTRANS

Utilisons notre fonction concat_data afin de **supprimer la colonne mère MTRANS** et de la **remplacer par les colonnes filles obtenues après labelbinarizer**

In [59]:
preprocessed_test = concat_data(preprocessed_test,df2, var_nom)

In [60]:
pd.set_option("display.max_columns",None)

In [61]:
preprocessed_test.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Automobile,Bike,Motorbike,Public_Transportation,Walking
0,20758,1.0,0.519293,1.697394,1.245441,1.0,1.0,3,3,2.0,0.0,3,0.0,1,0,1.0,0,0,0,1,0
1,20759,0.0,-0.503477,-1.145646,-0.831471,1.0,1.0,2,1,2.0,0.0,3,0.0,1,0,1.0,0,0,0,1,0
2,20760,0.0,0.363294,-0.649219,0.901712,1.0,1.0,3,3,2.0,0.0,3,0.0,0,0,1.0,0,0,0,1,0
3,20761,1.0,-0.507073,-1.682356,0.600254,1.0,1.0,2,3,2.0,0.0,3,0.0,0,0,1.0,0,0,0,1,0
4,20762,0.0,0.363294,-0.831954,0.644580,1.0,1.0,3,3,2.0,0.0,3,0.0,0,1,1.0,0,0,0,1,0


Comparons avec notre fichier train après avoir appliqué le preprocessing afin de voir que nous avons les memes colonnes :

In [62]:
final_train.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Automobile,Bike,Motorbike,Public_Transportation,Walking,NObeyesdad
0,0,1.0,0.093383,-0.000639,-0.235889,1.0,1.0,2,3,2.0,0.0,3,0.0,0,1,1.0,0,0,0,1,0,Overweight_Level_II
1,1,0.0,-1.023539,-1.603658,-1.173542,1.0,1.0,2,3,1.0,0.0,2,0.0,1,1,2.0,1,0,0,0,0,Normal_Weight
2,2,0.0,-1.023539,0.130604,-1.433298,1.0,1.0,2,1,2.0,0.0,2,0.0,1,2,2.0,0,0,0,1,0,Insufficient_Weight
3,3,0.0,-0.511670,0.122245,1.649491,1.0,1.0,3,3,2.0,0.0,2,0.0,1,1,1.0,0,0,0,1,0,Obesity_Type_III
4,4,1.0,1.341199,2.451877,0.225076,1.0,1.0,3,2,2.0,0.0,2,0.0,2,1,1.0,0,0,0,1,0,Overweight_Level_II


In [63]:
pd.set_option("display.max_columns",10)

- On peut voir qu'excepté la target présente uniquement dans le fichier train, toutes les colonnes sont sous le même format, nous allons pouvoir passer aux prédictions de notre modèles que nous avons entrainé sur le notebook 02_modélisation 
- Pour pouvoir utiliser le modèle, il faut **retirer la colonne id** sur lequel le modèle n'a pas été entrainé. Le format attendu par kaggle est un fichier contenant deux colonnes id:prédiction

In [64]:
identifiant = pd.DataFrame(preprocessed_test.id, columns=[preprocessed_test.id.name])
preprocessed_test.drop("id", axis = 1, inplace = True)

## Prédictions du modèle :

In [65]:
path_model = r"D:\Etude_Data_science\Kaggle_competition\03_Multi-Class Prediction of Obesity Risk\modeles\\"

### XGBC_GRID

In [66]:
with open(path_model+'XGBC_GRID.pkl', 'rb') as f:
    model = pickle.load(f)

In [67]:
prediction_num = model.predict(preprocessed_test)
prediction_num

array([3, 5, 4, ..., 0, 1, 3], dtype=int64)

In [68]:
### Transformation des prédictions aux données d'origine :
prediction = pd.DataFrame(target_encoder.inverse_transform(prediction_num),columns=["NObeyesdad"])
prediction.head()

,NObeyesdad
0,Obesity_Type_II
1,Overweight_Level_I
2,Obesity_Type_III
3,Obesity_Type_I
4,Obesity_Type_III


### Création du dataframe final pour soumettre à kaggle :

In [74]:
model_predict_df = pd.concat([identifiant, prediction], axis = 1)

In [70]:
model_predict_df.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


Pour kaggle, ici il faut mettre **l'id en tant qu'index**

In [71]:
prediction_kaggle = model_predict_df.set_index("id")

In [72]:
prediction_kaggle.head()

,NObeyesdad
id,
20758,Obesity_Type_II
20759,Overweight_Level_I
20760,Obesity_Type_III
20761,Obesity_Type_I
20762,Obesity_Type_III


### Enregistrement :

In [73]:
prediction_kaggle.to_csv("prediction_model_kaggle.csv")

In [77]:
a.set_index = False

In [78]:
a

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
...,...,...
13835,34593,Overweight_Level_II
13836,34594,Overweight_Level_I
13837,34595,Insufficient_Weight
13838,34596,Normal_Weight
